<a href="https://colab.research.google.com/github/yagnesh2004/Python/blob/main/medical_telegram_bot_2ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#AIMER Society - Indian Servers
!pip install pyTelegramBotAPI
!pip install openai
!pip install opencage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=eb2a68eb798ac917a71fd005724f4c8252b24972e4b9601ab8fc2e925cca4dd3
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [10]:
import telebot
import requests
from geopy.distance import geodesic

bot = telebot.TeleBot('6049992217:AAGnHYo1-_vYvKoqIkm0NixoBTrBjra0y34')

NOMINATIM_URL = "https://nominatim.openstreetmap.org/search"

def get_nearby_places(latitude, longitude, keyword, radius):
    params = {
        "q": keyword,
        "format": "json",
        "lat": latitude,
        "lon": longitude,
        "radius": radius,
    }

    response = requests.get(NOMINATIM_URL, params=params)
    if response.status_code == 200:
        results = response.json()
        places = []
        for result in results:
            if keyword in result.get("display_name", "").lower():
                place_address = result.get("display_name")
                place_coords = (float(result.get("lat")), float(result.get("lon")))
                distance = calculate_distance((latitude, longitude), place_coords)
                places.append({"address": place_address, "distance": distance})

        places.sort(key=lambda x: x["distance"])
        return places

    return None

def calculate_distance(coords1, coords2):
    return round(geodesic(coords1, coords2).kilometers, 2)

@bot.message_handler(commands=['start'])
def handle_start(message):
    response = "Welcome to the Emergency Locator Bot!\n\n"
    response += "To use this bot, follow these steps:\n"
    response += "1. Share your location using the location feature or send a Google Maps link.\n"
    response += "2. Select the type of service you're looking for.\n"
    response += "3. Enter the search radius in meters.\n"
    response += "4. Receive a list of nearby services and their distances.\n"
    bot.reply_to(message, response)

@bot.message_handler(content_types=['location'])
def handle_location(message):
    chat_id = message.chat.id
    latitude = message.location.latitude
    longitude = message.location.longitude

    markup = telebot.types.ReplyKeyboardMarkup(row_width=2, one_time_keyboard=True)
    markup.add(telebot.types.KeyboardButton('Hospitals'), telebot.types.KeyboardButton('Clinics'),
               telebot.types.KeyboardButton('Pharmacies'), telebot.types.KeyboardButton('Restaurants'))

    bot.send_message(chat_id, "Please select the type of service you're looking for:", reply_markup=markup)

    bot.register_next_step_handler(message, handle_service_selection, latitude, longitude)

def handle_service_selection(message, latitude, longitude):
    chat_id = message.chat.id
    service_type = message.text.lower()

    if service_type not in ['hospitals', 'clinics', 'pharmacies', 'restaurants']:
        bot.send_message(chat_id, "Invalid selection. Please try again.")
        return

    bot.send_message(chat_id, "Enter the search radius in meters:")

    bot.register_next_step_handler(message, handle_radius_input, latitude, longitude, service_type)

def handle_radius_input(message, latitude, longitude, service_type):
    chat_id = message.chat.id
    radius = int(message.text)

    try:
        places = get_nearby_places(latitude, longitude, service_type, radius)

        response = "Nearby {}:\n\n".format(service_type.capitalize())
        if places:
            for place in places:
                response += "- {} (Distance: {} km)\n".format(place["address"], place["distance"])
        else:
            response = "No {} found nearby.".format(service_type.capitalize())

        response += "\n\nIf you need any other services, feel free to search again."

        bot.send_message(chat_id, response)
    except Exception as e:
        bot.send_message(chat_id, "An error occurred while fetching the data. Please try again later.")

bot.polling()


2023-06-01 06:57:20,106 (__init__.py:1083 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 409. Description: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2023-06-01 06:57:20,110 (__init__.py:1085 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1073, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 108, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.10/dist-packages/telebot/util.py", line 90, in run
    task(*args, **kwargs)
  